# Import libraries

In [ ]:
!git clone https://github.com/amitkml/ERA-V1

Cloning into 'ERA-V1'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 98 (delta 7), reused 69 (delta 5), pack-reused 26
Unpacking objects: 100% (98/98), 85.55 MiB | 10.23 MiB/s, done.
Updating files: 100% (81/81), done.


In [ ]:
import os

In [ ]:
os.chdir("/content/ERA-V1/Session7")

In [ ]:
!pwd

/content/ERA-V1/Session7


In [ ]:
from __future__ import print_function
# BASE_DIR='/content/drive/MyDrive/regularization'
# %cd $BASE_DIR
from utils import train, test, model,plot_metrics,helper,data_loader
import torch

# Let's visualize some of the images
%matplotlib inline
import matplotlib.pyplot as plt
# import torchsummary

!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchsummary import summary


## Set the seed for reproducibility

In [ ]:
# Set Seed for reproducibility
SEED = 1
torch.manual_seed(SEED)

cuda = torch.cuda.is_available()
if cuda:
    torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if cuda else "cpu")


## Download data

In [ ]:
train_transforms, test_transforms = data_loader.data_augmentation()
train,test = data_loader.download_mnist_data(train_transforms, test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 63358485.16it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 116926345.39it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 67690040.10it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22924824.03it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Dataloader

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR,OneCycleLR
from utils import train as trn
from utils import test as tst

In [ ]:
BATCH_SIZE_GPU = 64
BATCH_SIZE_CPU = 64

dataloader_args = dict(shuffle=True, batch_size=BATCH_SIZE_GPU, num_workers=1, pin_memory=True) if cuda else dict(shuffle=True, batch_size=BATCH_SIZE_CPU)
train_loader, test_loader = data_loader.dataloader(train, test,dataloader_args)

## Variable Initialization

In [ ]:
exp_metrics={}
EPOCH=15
dropout=0.03

## Exp -5
Target:

    Have network parameter further less than 8K. Have increased the LR.

Results:

    Parameters: 5,368
    Best Train Accuracy: 98.16
    Best Test Accuracy: 99.25

Analysis:

    Network accuracy has been reduced a bit
    May need to play a bit more with LR


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict



class Net5X(nn.Module):
    def __init__(self,norm_type='BN',dropout_value = 0.01):
        super(Net5X, self).__init__()
        self.conv1 = self.conv2d(1, 8, 3,norm_type,dropout_value,2)
        self.conv2 = self.conv2d(8, 16, 3,norm_type,dropout_value,4)

        #Transition Block
        self.trans1 = nn.Sequential(

            nn.MaxPool2d(2, 2), #  Input 24x24 output 12x12 RF : 6x6
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0, bias=False)  # Input 12x12 output 12x12 RF : 6x6
        )

        self.conv3 = self.conv2d(8, 16, 3,norm_type,dropout_value,4) ## op: 10x10
        self.conv4 = self.conv2d(16, 16, 3,norm_type,dropout_value,4) ## op 8x8
        self.trans2 = nn.Sequential(

            nn.MaxPool2d(2, 2), #  Input 8x8 output 4x4
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1), padding=0, bias=False)  # Input 4x4 , op: 4x4
        )
        # self.conv5 = self.conv2d(16, 16, 3,norm_type,dropout_value,4) ## Input 4x4 , op: 3X3
        self.avgpool2d = nn.AvgPool2d(kernel_size=4)
        self.conv6 =  self.conv2d_last(8, 16, 1,norm_type,dropout_value,4)
        self.conv7 = nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False)
    def conv2d_last(self, in_channels, out_channels, kernel_size, norm_type, dropout,num_of_groups):
            conv = nn.Sequential(OrderedDict([
                ('conv2d',nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=0, bias=False)),
                # ('Relu', nn.ReLU()),
                ('GroupNorm',nn.GroupNorm(num_of_groups,out_channels)),
                # ('Dropout',nn.Dropout(dropout))
            ]))
            return conv

    def conv2d(self, in_channels, out_channels, kernel_size, norm_type, dropout,num_of_groups):
        if norm_type == "BN":
         conv = nn.Sequential(OrderedDict([
                ('conv2d', nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,padding=0, bias=False)),
                ('Relu', nn.ReLU()),
                ('BatchNorm',nn.BatchNorm2d(out_channels)),
                ('Dropout', nn.Dropout(dropout))
         ]))
        elif norm_type == "LN":
            conv = nn.Sequential(OrderedDict([
                ('conv2d',nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=0, bias=False)),
                ('Relu', nn.ReLU()),
                ## When number of groups is 1, its layernorm
                ('LayerNorm',nn.GroupNorm(1,out_channels)),
                ('Dropout',nn.Dropout(dropout))
            ]))
        elif norm_type == "GN":
            conv = nn.Sequential(OrderedDict([
                ('conv2d',nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=0, bias=False)),
                ('Relu', nn.ReLU()),
                ('GroupNorm',nn.GroupNorm(num_of_groups,out_channels)),
                ('Dropout',nn.Dropout(dropout))
            ]))
        else:
            conv = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=0, bias=False),
                nn.ReLU(),
                nn.Dropout(dropout)
            )

        return conv

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.trans1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.trans2(x)
        # x = self.conv5(x)
        x = self.avgpool2d(x)
        x = self.conv6(x)
        x = self.conv7(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [ ]:
def experiments5X(train_loader, test_loader, norm_type, l1_factor, l2_factor, dropout, epochs):

    train_losses = []
    test_losses = []
    train_accuracy = []
    test_accuracy = []

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    model = Net5X(norm_type, dropout).to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.025, momentum=0.97,weight_decay=l2_factor)
    scheduler = OneCycleLR(optimizer, max_lr=0.025,epochs=epochs,steps_per_epoch=len(train_loader))
    epochs = epochs

    for epoch in range(1, epochs + 1):
        print(f'Epoch {epoch}:')
        trn.train(model, device, train_loader, optimizer, epoch, train_accuracy, train_losses, l1_factor,scheduler)
        tst.test(model, device, test_loader,test_accuracy,test_losses)

    return (train_accuracy,train_losses,test_accuracy,test_losses),model

In [ ]:
experiment_name='LayerNorm_5x'
norm_type='LN'
l1_factor=0
l2_factor=0

In [ ]:
(exp_metrics[experiment_name]), mode5x  = experiments5X(train_loader, test_loader,'LN',l1_factor,l2_factor,dropout,EPOCH)

Epoch 1:


Loss=0.29722657799720764 Batch_id=937 Accuracy=66.13: 100%|██████████| 938/938 [01:30<00:00, 10.38it/s]



Test set: Average loss: 0.2213, Accuracy: 9406/10000 (94.06%)

Epoch 2:


Loss=0.1429508924484253 Batch_id=937 Accuracy=91.02: 100%|██████████| 938/938 [01:24<00:00, 11.04it/s]



Test set: Average loss: 0.1197, Accuracy: 9649/10000 (96.49%)

Epoch 3:


...


Test set: Average loss: 0.0303, Accuracy: 9917/10000 (99.17%)

Epoch 14:


Loss=0.006646742112934589 Batch_id=937 Accuracy=98.15: 100%|██████████| 938/938 [01:22<00:00, 11.33it/s]



Test set: Average loss: 0.0297, Accuracy: 9921/10000 (99.21%)

Epoch 15:


Loss=0.15876784920692444 Batch_id=937 Accuracy=98.16: 100%|██████████| 938/938 [01:24<00:00, 11.14it/s]



Test set: Average loss: 0.0287, Accuracy: 9925/10000 (99.25%)



In [ ]:
helper.model_summary(mode5x,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
         GroupNorm-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
         GroupNorm-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
        GroupNorm-13           [-1, 16, 10, 10]              32
          Dropout-14           [-1, 16,